In [3]:
import bempp.api
import numpy as np
import scipy
from numba import objmode
import numba
import math
%matplotlib inline
import matplotlib.pyplot as plt
from bempp.api.operators.far_field import helmholtz as helmholtz_farfield
#bempp.api.show_available_platforms_and_devices()
#bempp.api.set_default_device(0, 0)

In [3]:
def normalized_spherical_harmonics(p, q, x):
    """Spherical Harmonic function of degree q"""
    azimuth = np.arctan2(x[1],x[0])
    polar = np.arccos(x[2]/np.linalg.norm(x))
    if p >= 0:
        return ((-1)**p) * scipy.special.sph_harm(p,q,azimuth,polar)
    else:
        return scipy.special.sph_harm(-p,q,azimuth,polar)*np.exp(1j*2*p*azimuth)

def spherical_bessel_function(q,x):
    """Spherical Bessel function of degree q"""
    r = np.linalg.norm(x)
    return np.sqrt(np.pi/(2*k*r))*scipy.special.jv(q+0.5, k*r)


def spherical_hankel_function(q,x):
    """Spherical hankel function of degree q"""
    r = np.linalg.norm(x)
    return np.sqrt(np.pi/(2*k*r))*scipy.special.hankel1(q+0.5, k*r)

def regular_spherical_wavefunctions(p,q,x):
    """Regular Spherical Wavefunction"""
    return spherical_bessel_function(q,x)*normalized_spherical_harmonics(p,q,x)

def radiating_spherical_wavefunctions(p,q,x):
    """Radiating Spherical Wavefunction"""
    return spherical_hankel_function(q,x)*normalized_spherical_harmonics(p,q,x)

In [4]:
def coeff_sph_expansion(p,q):
    """Analytical formula of the coefficients of the spherical expansions of plane waves"""
    coeff = 4*np.pi*((1j)**q)*normalized_spherical_harmonics(-p,q,d)/(-1)**(abs(p))
    return coeff

In [6]:
def approx_scat(x):
    appro = 0
    i = 0
    for q in range(deg+1):
        for p in range(-q,q+1):
            appro += coeff_sca[i] * radiating_spherical_wavefunctions(p,q,x)
            i += 1
    return appro

In [9]:
d = [1,0,0] # wave's travel direction
k = 5 # wavenumber
eta = k
error_list = []

In [15]:
for deg in [5,7,9,11,13,15]:
    for h in [0.05,0.07,0.09,0.11]:
        #sphere that encloses the scatterer
        sphere = bempp.api.shapes.sphere(r = 2, origin=(0,0,0), h = h)
        vert_sphere = sphere.vertices
        space_sphere = bempp.api.function_space(sphere, 'P', 1)

        # scatterer: sphere
        dom_cube = bempp.api.shapes.cube(length = (2*2)/1.8, origin=(-2/1.8,-2/1.8,-2/1.8), h = h)
        vert_cube = dom_cube.vertices
        space_cube = bempp.api.function_space(dom_cube,'P',1)

        identity = bempp.api.operators.boundary.sparse.identity(space_cube, space_cube,space_cube)
        slp = bempp.api.operators.boundary.helmholtz.single_layer(space_cube,space_cube,space_cube,k)
        dlp = bempp.api.operators.boundary.helmholtz.double_layer(space_cube,space_cube,space_cube,k)

        lhs = 0.5 * identity + dlp - 1j* eta *slp

        coeff_scat_fields = []
        for q in range(deg+1):
            for p in range(-q,q+1):
                @bempp.api.complex_callable
                def dirichlet_fun(x,n,domain_index,result):
                        with objmode():
                            result[0] = - regular_spherical_wavefunctions(p,q,x)

                rhs_fun = bempp.api.GridFunction(space_cube, fun = dirichlet_fun)
                field, info = bempp.api.linalg.gmres(lhs, rhs_fun, tol=1E-5)
                slp_pot = bempp.api.operators.potential.helmholtz.single_layer(space_cube, vert_sphere, k)
                dlp_pot = bempp.api.operators.potential.helmholtz.double_layer(space_cube, vert_sphere,k)
                coeff_scat_fields.append(dlp_pot.evaluate(field) - 1j * eta * slp_pot.evaluate(field))

        coeff_sph_wf = []
        for q in range(deg+1):
            for p in range(-q,q+1):
                @bempp.api.complex_callable
                def basis_fun(x,n,domain_index,result):
                    with objmode():
                        result[0] = radiating_spherical_wavefunctions(p,q,x)

                basis_grid = bempp.api.GridFunction(space_sphere,fun=basis_fun)
                basis_coeff = basis_grid.coefficients
                coeff_sph_wf.append(basis_coeff)

        mass_mat = bempp.api.operators.boundary.sparse.identity(space_sphere,space_sphere,space_sphere).weak_form().A

        '''Construct list of b'''
        b = np.zeros(((deg+1)**2, (deg+1)**2),dtype = complex)
        for i in range((deg+1)**2):
            for j in range((deg+1)**2):
                b[i,j] = coeff_scat_fields[j] @ mass_mat @ coeff_sph_wf[i]

        '''Construct A'''
        A = np.zeros(((deg+1)**2, (deg+1)**2),dtype = complex)
        for i in range((deg+1)**2):
            for j in range((deg+1)**2):
                A[i,j] = coeff_sph_wf[j] @ mass_mat @ coeff_sph_wf[i]

        T_matrix = np.zeros(((deg+1)**2, (deg+1)**2),dtype = complex)
        for i in range((deg+1)**2):
            T_matrix[:,i] = np.linalg.solve(A,b[:,i])
        
        coeff_inc = []
        for q in range(deg+1):
            for p in range(-q, q+1):
                coeff_inc.append(coeff_sph_expansion(p,q))

        coeff_sca = T_matrix @ coeff_inc

        @bempp.api.complex_callable
        def approx_scat_fuc(x,n,domain_index,result):
            with objmode():
                result[0] = approx_scat(x)   
        approx_grid = bempp.api.GridFunction(space_sphere,fun=approx_scat_fuc)
        approx_coeff = approx_grid.coefficients

        '''compute error'''
        @bempp.api.complex_callable
        def incident_field(x,n,domain_index,result):
                with objmode():
                    result[0] = - np.exp(1j*k*x[0])

        rhs_fun_exact = bempp.api.GridFunction(space_cube, fun = incident_field)
        field_exact, info_exact = bempp.api.linalg.gmres(lhs, rhs_fun_exact, tol=1E-5)

        exact_coeff = dlp_pot.evaluate(field_exact) - 1j * eta * slp_pot.evaluate(field_exact)

        err = np.sqrt((approx_coeff - exact_coeff) @ mass_mat @ np.transpose(np.conj(approx_coeff - exact_coeff)))/np.sqrt(exact_coeff @ mass_mat @ np.transpose(np.conj(exact_coeff)))

        error_list.append(err)
        print(h,deg,err)

0.05 5 [[0.70896114+5.13478557e-18j]]
0.07 5 [[0.70848091+3.2515815e-18j]]
0.09 5 [[0.7078652-5.40653649e-18j]]
0.11 5 [[0.70714849+6.190203e-20j]]
0.05 7 [[0.38638149-3.07319683e-18j]]
0.07 7 [[0.38574021-1.1504717e-18j]]
0.09 7 [[0.38492947-2.86079854e-18j]]
0.11 7 [[0.38400125-3.61377058e-18j]]
0.05 9 [[0.12602519-1.36643988e-19j]]
0.07 9 [[0.12556999+2.49276901e-19j]]
0.09 9 [[0.12498173-2.91525873e-19j]]
0.11 9 [[0.12433107-1.11184916e-20j]]


KeyboardInterrupt: 

### Compute the error